# Modelos de Regresión Lineal

## Análisis Exploratorio de Datos 

### Carga del dataset

In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
train_file_path = "data/train_set.csv"
df_train = pd.read_csv(train_file_path)
print(df_train.head())

   MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0          20       RL         70.0     8400   Pave   NaN      Reg   
1          60       RL         59.0     7837   Pave   NaN      IR1   
2          30       RL         67.0     8777   Pave   NaN      Reg   
3          50       RL         60.0     7200   Pave   NaN      Reg   
4          50       RL         50.0     5000   Pave  Pave      Reg   

  LandContour Utilities LotConfig  ... PoolArea PoolQC  Fence MiscFeature  \
0         Lvl    AllPub    Inside  ...        0    NaN    NaN         NaN   
1         Lvl    AllPub    Inside  ...        0    NaN    NaN         NaN   
2         Lvl    AllPub    Inside  ...        0    NaN  MnPrv         NaN   
3         Lvl    AllPub    Corner  ...        0    NaN  MnPrv         NaN   
4         Lvl    AllPub    Inside  ...        0    NaN    NaN         NaN   

  MiscVal MoSold  YrSold  SaleType  SaleCondition  SalePrice  
0       0      6    2010        WD         Normal    

In [43]:
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,145000
1,60,RL,59.0,7837,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,5,2009,WD,Normal,178000
2,30,RL,67.0,8777,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,5,2008,WD,Normal,85000
3,50,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,NaN,MnPrv,NaN,0,6,2007,WD,Normal,175000
4,50,RL,50.0,5000,Pave,Pave,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,127000


### Limpieza de datos

Manejo de Valores Nulos - Se  obtienen la cantidad de valores nulos y el porcentaje para determinar que datos se pueden eliminar y como manejar aquellos que representan una gran porcion de los datos

In [44]:
def relevant_null_value_percentage(df, umbral):
    na_not_missing = [
        "Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2",
        "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond",
        "PoolQC", "Fence", "MiscFeature"
    ]
    df_cleaning = df.isnull().sum().reset_index()
    df_cleaning.columns = ["Columnas", "Valores Nulos"]
    df_cleaning["Porcentaje de Valores Nulos"] = (df_cleaning["Valores Nulos"] / len(df)) * 100
    df_cleaning["Tipo de Nulo"] = df_cleaning["Columnas"].apply(
        lambda x: "No Aplica" if x in na_not_missing else "Dato Faltante"
    )
    df_cleaning = df_cleaning[df_cleaning["Porcentaje de Valores Nulos"] > umbral]
    df_cleaning = df_cleaning.sort_values(by="Porcentaje de Valores Nulos", ascending=False)

    return df_cleaning

In [45]:
df_nulos = relevant_null_value_percentage(df_train, 0)
print(df_nulos)

        Columnas  Valores Nulos  Porcentaje de Valores Nulos   Tipo de Nulo
71        PoolQC           1162                    99.486301      No Aplica
73   MiscFeature           1122                    96.061644      No Aplica
5          Alley           1094                    93.664384      No Aplica
72         Fence            935                    80.051370      No Aplica
24    MasVnrType            683                    58.476027  Dato Faltante
56   FireplaceQu            547                    46.832192      No Aplica
2    LotFrontage            217                    18.578767  Dato Faltante
57    GarageType             64                     5.479452      No Aplica
58   GarageYrBlt             64                     5.479452  Dato Faltante
59  GarageFinish             64                     5.479452      No Aplica
62    GarageQual             64                     5.479452      No Aplica
63    GarageCond             64                     5.479452      No Aplica
30      Bsmt

Para los columnas que presenten un porcentaje a 5% se deben trabajar los datos y para aquellas con un porcentaje menor a 5% se pueden eliminar esas filas con los datos nulos.

In [46]:
df_train_cleaned = df_train.copy()
# 1. Manejo de valores en MasVnrType con la moda (valor más frecuente)
mode_masvnrtype = df_train_cleaned["MasVnrType"].mode()[0]  # Obtener la moda
df_train_cleaned.loc[:, "MasVnrType"] = df_train_cleaned["MasVnrType"].fillna(mode_masvnrtype)

# 2. Manejo de  valores en LotFrontage con la mediana por Neighborhood
df_train_cleaned.loc[:, "LotFrontage"] = df_train_cleaned.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median())
)

# 3. Imputar valores en GarageYrBlt con YearBuilt
df_train_cleaned.loc[:, "GarageYrBlt"] = df_train_cleaned["GarageYrBlt"].fillna(df_train_cleaned["YearBuilt"])

missing_after_imputation = df_train_cleaned[["MasVnrType", "LotFrontage", "GarageYrBlt"]].isnull().sum()

In [47]:
print("Valores nulos después de la imputación:")
print(missing_after_imputation)

Valores nulos después de la imputación:
MasVnrType     0
LotFrontage    0
GarageYrBlt    0
dtype: int64


Eliminar filas con valores null (menores al 5%)

In [48]:
df_train_cleaned = df_train_cleaned.dropna()
missing_after_drop = df_train_cleaned.isnull().sum()

In [49]:
print("Valores nulos después de eliminar filas con datos faltantes:")
print(missing_after_drop)

Valores nulos después de eliminar filas con datos faltantes:
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 80, dtype: int64


In [ ]:
# Cargar el conjunto de datos nuevamente para asegurarnos de tener la versión original
train_file_path = "/mnt/data/train_set.csv"
df_train = pd.read_csv(train_file_path)

# 1. Identificar variables categóricas
categorical_vars = df_train.select_dtypes(include=['object']).columns

# 2. Decidir el tipo de transformación
# Label Encoding para variables con orden lógico
label_enc_vars = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 
                  'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']

# One-Hot Encoding para el resto
one_hot_enc_vars = [col for col in categorical_vars if col not in label_enc_vars]

# 3. Aplicar Label Encoding
label_encoders = {}
for col in label_enc_vars:
    le = LabelEncoder()
    # Convertimos a string para evitar problemas con NaN
    df_train[col] = le.fit_transform(df_train[col].astype(str))
    label_encoders[col] = le

# 4. Aplicar One-Hot Encoding
df_train_one_hot_encoded = pd.get_dummies(df_train[one_hot_enc_vars])

# 5. Unir los resultados al conjunto de datos original
df_train_transformed = pd.concat([df_train.drop(columns=one_hot_enc_vars), df_train_one_hot_encoded], axis=1)

# Mostrar resultados
{
    
    "Total de Columnas Antes de Transformación": len(df_train.columns),
    "Total de Columnas Después de Transformación": df_train_transformed.shape[1],
    "Ejemplo de Columnas con One-Hot Encoding": df_train_one_hot_encoded.columns[:5].tolist()
}


{'One-Hot Encoding Aplicado a': ['MSZoning',
  'Street',
  'Alley',
  'LotShape',
  'LandContour',
  'Utilities',
  'LotConfig',
  'LandSlope',
  'Neighborhood',
  'Condition1',
  'Condition2',
  'BldgType',
  'HouseStyle',
  'RoofStyle',
  'RoofMatl',
  'Exterior1st',
  'Exterior2nd',
  'MasVnrType',
  'Foundation',
  'BsmtExposure',
  'BsmtFinType1',
  'BsmtFinType2',
  'Heating',
  'CentralAir',
  'Electrical',
  'Functional',
  'GarageType',
  'GarageFinish',
  'PavedDrive',
  'Fence',
  'MiscFeature',
  'SaleType',
  'SaleCondition'],
 'Total de Columnas Después de Transformación': 252}

### Análisis descriptivo

### Visualización de datos

### Pruebas de normalidad y correlación

### Análisis de agrupamiento

## Ingeniería de Características

### Selección de variables relevantes

### Creación de nuevas características

### Análisis de multicolinealidad